In [1]:
from Functions import PlotFunctions
from Functions import General

import time
from tqdm import tqdm
import pandas as pd
from datetime import datetime
import os
from bs4 import BeautifulSoup
import quandl
from time import mktime


from ipdb import set_trace

import numpy as np
path='c:\\Users\\15764\\Documents\\Datasets\\intraQuarter\\'

In [2]:
#need to extract only axusition

In [ ]:
def GetStats(gathers = ['Total Debt/Equity (mrq)'],path='', limit = None):
    
    #INIT Data##################
    columns = ['Date',
                'Unix',
                'Ticker']
    columns.extend(gathers)
    
    print(*gathers,sep=',')
    stock_files = path+ '_KeyStats'
    stock_list = [x[0] for x in os.walk(stock_files)]
    df = pd.DataFrame(columns=columns)
    
    #print(stacks_list)
    amount_of_files = len(stock_list[1:])
    if limit:
        print("Limit set to:", limit)
    else:
        print("Will Crowl All dataset")
    counter = -1
    
    ##################
    
    #Go through Each Ticker (folder)
    for each_dir in tqdm(stock_list[1:]):
        counter +=1
        if limit==counter:
            break
        ticker = each_dir.split('\\')[-1]
        print('Start working with: ',ticker)
        each_file = os.listdir(each_dir)
        
        if len(each_file)>0:
            #Get all historic data 
            for file in tqdm(each_file):
                
                date_stamp = datetime.strptime(file,'%Y%m%d%H%M%S.html')
                unix_time = time.mktime(date_stamp.timetuple())
                full_path_file = each_dir +'\\'+file
                with open(full_path_file,'r') as f:
                    source =f.read()
                soup = BeautifulSoup(source, 'html.parser')
                data = []
                rows = soup.find_all('tr')
                for row in rows:
                    cols = row.find_all('td')
                    data.extend([cell.text for cell in cols])
                try:
                    values = {}

                    for gather in  gathers:
                        try:
                            values[gather] = float(data[data.index(gather+':')+1])
                        except Exception as e: 
                            values[gather] = -99999.0
                    
                    #Get stock price for spesific date
                    new_row = {**{'Date':date_stamp,
                                    'Unix':unix_time,
                                    'Ticker':ticker,
                                   },**values}
                    df = df.append(new_row,ignore_index=True)
                except Exception as e:
                    pass
    return df

In [ ]:
df = GetStats(path=path,limit=2)

Total Debt/Equity (mrq)
Limit set to: 2


  0%|                                                                                          | 0/560 [00:00<?, ?it/s]

Start working with:  a



  0%|                                                                                           | 0/18 [00:00<?, ?it/s]

> <ipython-input-3-cbfa731074be>(35)GetStats()
     34             #Get all historic data
---> 35             for file in tqdm(each_file):
     36 



In [28]:
df.head()

,Date,Unix,Ticker,Total Debt/Equity (mrq)
0,2004-01-30 19:01:02,1.075511e+09,a,NaN
1,NaT,NaN,NaN,0.407
2,2004-04-13 04:07:11,1.081847e+09,a,NaN
3,NaT,NaN,NaN,0.375
4,2004-12-13 21:37:34,1.102995e+09,a,NaN


In [ ]:
columns = ['Date',
                'Unix',
                'Ticker',
                gather,
                'Stock Price',
                'Stock Change',
                'SNP500 Value',
                'SNP500 Change',
              'YtY Stock Price Change',
              'YtY SNP500 Value Change',
              'Absolute Stock Perfomance',
              'YtY Stock Perfomance',
              'Absolute Difference',
              'YtY Difference', 
              'Status'] #is the stack is overperform or outperfrorm

df = df.reset_index()
starting_stock_value = False
starting_sp_500_value = False
snp500 = pd.read_csv(r'c:\Users\15764\Documents\Datasets\intraQuarter\YAHOO-INDEX_GSPC.csv', index_col=0 )

if not starting_stock_value:
    starting_stock_value = stock_price

if not starting_sp_500_value:
    starting_sp_500_value = snp500_value

try:
    snp500_data = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d')
    row = snp500[snp500.index==snp500_data]
    snp500_value = float(row['Adj Close'])
except:
    snp500_data = datetime.fromtimestamp(unix_time-259200).strftime('%Y-%m-%d')
    row = snp500[snp500.index==snp500_data]
    snp500_value = float(row['Adj Close'])


def ytypchange(row):
    
    if row[columns[0]] == df.loc[0,columns[0]]:
        return 0
    else:
        return 100*(df.loc[row['index'],'Stock Price'] -df.loc[row['index']-1,'Stock Price']) / df.loc[row['index']-1,'Stock Price']
def ytysnpchange(row):
    if row[columns[0]] == df.loc[0,columns[0]]:
        return 0
    else:
        return 100 *(df.loc[row['index'],'SNP500 Value'] -df.loc[row['index']-1,'SNP500 Value']) / df.loc[row['index']-1,'SNP500 Value']

def calctabsperf(row):
    if row[columns[5]] - row[columns[7]] > 0:
        return 1
    else:
        return 0

def calcttytperf(row):
    if row[columns[8]] - row[columns[9]] > 0:
        return 1
    else:
        return 0

ytypchange_data = df.apply(ytypchange,axis=1) #'YtY Stock Price Change',
ytysnpchange_data = df.apply(ytysnpchange,axis=1) #'YtY SNP500 Value Change'


calctabsperf_data = df.apply(calctabsperf,axis=1) #'Absolute Stock Perfomance',
calcttytperf_data = df.apply(calcttytperf,axis=1) #'YtY Stock Perfomance'
abs_difference_data = df.apply(lambda x : x[columns[5]]-x[columns[7]],axis=1)#'Absolute Difference'
YtYDifference = df.apply(lambda x : x[columns[8]]-x[columns[9]],axis=1)#'YtY Difference'
Status = np.where((stock_change - snp500_change) >0,1,0) #1 - outperform, 0 - ounderperform


stock_price = quandl_stocks_host_price(symbol=ticker,date=snp500_data)

stock_change = 100*(stock_price -starting_stock_value) / starting_stock_value
snp500_change = 100*(snp500_value - starting_sp_500_value) / starting_sp_500_value
# print(stock_price)


df = df.append({'Date':date_stamp, #'Data'
                'Unix':unix_time, #'Unix',
                'Ticker':ticker, #'Ticker',
                gather:value,# gather,
                'Stock Price':stock_price, #'Stock Price',
                'Stock Change': stock_change, #'Stock % Change',
                'SNP500 Value':snp500_value, #'SNP500 Value',
                'SNP500 Change': snp500_change,#'SNP500 % Cnange',
                'YtY Stock Price Change':ytypchange_data,
                'YtY SNP500 Value Change' :ytysnpchange_data,
                'Absolute Stock Perfomance':calctabsperf_data,
                'Absolute Difference':abs_difference_data,
                'YtY Difference': YtYDifference,
                'Status': Status
               }


            ,ignore_index=True) #'SNP500 Change',



df.drop('index',axis=1,inplace=True)
save = gather.replace(' ','').replace('(','').replace(')','').replace('/','')+('.csv')
print('Will save to file: ',save)
df.to_csv(save,index=False)
return df

